In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

In [2]:
import logging
from oct2py import Oct2Py, get_log
oc = Oct2Py(logger=get_log())
oc.addpath('./src')
oc.logger = get_log('new_log')
oc.logger.setLevel(logging.INFO)

In [3]:
def Hbeta(D, beta):
    P = np.exp(-D * beta)
    sumP = np.sum(P)
    H = np.log(sumP) + beta * np.sum(np.multiply(D, P)) / sumP
    P = P / sumP
    return H, P

In [4]:
D = np.random.random(784)
beta = 1 + np.random.randint(64)
%time Ho, Po = oc.Hbeta(D, beta)
%time Hp, Pp = Hbeta(D, beta)
print(np.allclose(Ho, Hp) and np.allclose(Po, Pp))

ans =  2
'Hbeta' is a function from the file /Users/kyle/Documents/Learning/Parametric t-SNE/src/Hbeta.m

Hbeta computes the Gaussian kernel values given a vector of
 squared Euclidean distances, and the precision of the Gaussian kernel.
 The function also computes the perplexity of the distribution.


Additional help for built-in functions and operators is
available in the online version of the manual.  Use the command
'doc <topic>' to search the manual index.

Help and information about Octave is also available on the WWW
at http://www.octave.org and via the help@octave.org
mailing list.
CPU times: user 88.5 ms, sys: 43.8 ms, total: 132 ms
Wall time: 241 ms
CPU times: user 157 µs, sys: 37 µs, total: 194 µs
Wall time: 219 µs
True


In [4]:
def x2p(X, u=15, tol=1e-4, print_iter=500, max_tries=50, verbose=0):
    # Initialize some variables
    n = X.shape[0]                     # number of instances
    P = np.zeros((n, n))               # empty probability matrix
    beta = np.ones(n)                  # empty precision vector
    logU = np.log(u)                   # log of perplexity (= entropy)
    
    # Compute pairwise distances
    if verbose > 0: print('Computing pairwise distances...')
    sum_X = np.sum(np.square(X), axis=1)
    # note: translating sum_X' from matlab to numpy means using reshape to add a dimension
    D = sum_X + sum_X[:,None] + -2 * X.dot(X.T)

    # Run over all datapoints
    if verbose > 0: print('Computing P-values...')
    for i in range(n):
        
        if verbose > 1 and print_iter and i % print_iter == 0:
            print('Computed P-values {} of {} datapoints...'.format(i, n))
        
        # Set minimum and maximum values for precision
        betamin = float('-inf')
        betamax = float('+inf')
        
        # Compute the Gaussian kernel and entropy for the current precision
        indices = np.concatenate((np.arange(0, i), np.arange(i + 1, n)))
        Di = D[i, indices]
        H, thisP = Hbeta(Di, beta[i])
        
        # Evaluate whether the perplexity is within tolerance
        Hdiff = H - logU
        tries = 0
        while abs(Hdiff) > tol and tries < max_tries:
            
            # If not, increase or decrease precision
            if Hdiff > 0:
                betamin = beta[i]
                if np.isinf(betamax):
                    beta[i] *= 2
                else:
                    beta[i] = (beta[i] + betamax) / 2
            else:
                betamax = beta[i]
                if np.isinf(betamin):
                    beta[i] /= 2
                else:
                    beta[i] = (beta[i] + betamin) / 2
            
            # Recompute the values
            H, thisP = Hbeta(Di, beta[i])
            Hdiff = H - logU
            tries += 1
        
        # Set the final row of P
        P[i, indices] = thisP
        
    if verbose > 0: 
        print('Mean value of sigma: {}'.format(np.mean(np.sqrt(1 / beta))))
        print('Minimum value of sigma: {}'.format(np.min(np.sqrt(1 / beta))))
        print('Maximum value of sigma: {}'.format(np.max(np.sqrt(1 / beta))))
    
    return P, beta

In [6]:
X = np.random.random((512, 32))
%time Po, betao = oc.x2p(X)
%time Pp, betap = x2p(X, verbose=2)
print(np.allclose(Po, Pp) and np.allclose(betao.flatten(), betap.flatten()))

ans =  2
'x2p' is a function from the file /Users/kyle/Documents/Learning/Parametric t-SNE/src/x2p.m

X2P Identifies appropriate sigma's to get kk NNs up to some tolerance 

   [P, beta] = x2p(xx, kk, tol)
 
 Identifies the required precision (= 1 / variance^2) to obtain a Gaussian
 kernel with a certain uncertainty for every datapoint. The desired
 uncertainty can be specified through the perplexity u (default = 15). The
 desired perplexity is obtained up to some tolerance that can be specified
 by tol (default = 1e-4).
 The function returns the final Gaussian kernel in P, as well as the 
 employed precisions per instance in beta.



Additional help for built-in functions and operators is
available in the online version of the manual.  Use the command
'doc <topic>' to search the manual index.

Help and information about Octave is also available on the WWW
at http://www.octave.org and via the help@octave.org
mailing list.
Computing pairwise distances...
Computing P-values...
Computed P

In [5]:
def compute_joint_probabilities(samples, batch_size=5000, d=2, perplexity=30, tol=1e-5, verbose=0):
    v = d - 1
    
    # Initialize some variables
    n = samples.shape[0]
    batch_size = min(batch_size, n)
    
    # Precompute joint probabilities for all batches
    if verbose > 0: print('Precomputing P-values...')
    batch_count = int(n / batch_size)
    P = np.zeros((batch_count, batch_size, batch_size))
    for i, start in enumerate(range(0, n - batch_size + 1, batch_size)):   
        curX = samples[start:start+batch_size]                   # select batch
        P[i], beta = x2p(curX, perplexity, tol, verbose=verbose) # compute affinities using fixed perplexity
        P[i][np.isnan(P[i])] = 0                                 # make sure we don't have NaN's
        P[i] = (P[i] + P[i].T) # / 2                             # make symmetric
        P[i] = P[i] / P[i].sum()                                 # obtain estimation of joint probabilities
        P[i] = np.maximum(P[i], np.finfo(P[i].dtype).eps)

    return P

In [8]:
from keras.datasets import mnist

%time (X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

CPU times: user 1.34 s, sys: 607 ms, total: 1.94 s
Wall time: 1.95 s
(60000, 'train samples')
(10000, 'test samples')


In [9]:
%time P = compute_joint_probabilities(X_train, verbose=2)

Precomputing P-values...
Computing pairwise distances...
Computing P-values...
Computed P-values 0 of 5000 datapoints...
Computed P-values 500 of 5000 datapoints...
Computed P-values 1000 of 5000 datapoints...
Computed P-values 1500 of 5000 datapoints...
Computed P-values 2000 of 5000 datapoints...
Computed P-values 2500 of 5000 datapoints...
Computed P-values 3000 of 5000 datapoints...
Computed P-values 3500 of 5000 datapoints...
Computed P-values 4000 of 5000 datapoints...
Computed P-values 4500 of 5000 datapoints...
Mean value of sigma: 2.21039597265
Minimum value of sigma: 1.14651898224
Maximum value of sigma: 3.42356224164
Computing pairwise distances...
Computing P-values...
Computed P-values 0 of 5000 datapoints...
Computed P-values 500 of 5000 datapoints...
Computed P-values 1000 of 5000 datapoints...
Computed P-values 1500 of 5000 datapoints...
Computed P-values 2000 of 5000 datapoints...
Computed P-values 2500 of 5000 datapoints...
Computed P-values 3000 of 5000 datapoints...

In [34]:
# %time np.save('P.npy', P)
# %time P = np.load('P.npy')

CPU times: user 84.5 ms, sys: 3.32 s, total: 3.41 s
Wall time: 4.48 s
